### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [4]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [5]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [8]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [9]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [10]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [11]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [12]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [13]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [14]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], dtype=int64)

In [15]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [16]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [17]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [18]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [19]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [20]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

#### Carga de librerías necesarias 

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.datasets import fetch_20newsgroups

import numpy as np

In [22]:
# Bajamos el dataset de forma separada para trainy test
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [23]:
# Instanciamos el vectorizador TF-IDF (Term frequency-Inverse term frequency)
tfidfvect = TfidfVectorizer()

# Entrenamos el vectorizador, calculando el diccionario, el vector IDF y obteniendo
# los vectores TF-IDF
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [24]:
from tabulate import tabulate

RANDOM_STATE = 41
N_DOCS_TO_COMPARE = 5
N_TOP_SCORE = 5

default_rng = np.random.default_rng(seed=RANDOM_STATE)
idxs = default_rng.choice(
    X_train.shape[0], N_DOCS_TO_COMPARE, replace=False, p=None, axis=0, shuffle=True)

similarities = cosine_similarity(X_train[idxs], X_train)
# En la matriz similarities para cada fila hay, al menos, un elemento máximo teóricamente igual
# a 1, pero computacionalmente, por cuestiones de representación, cercano a uno. Si sólo hay uno
# ese índice hará refernecia al documento que se está utilizando como referencia.
# También puede haber más de una aparición de este valor máximo, de hecho, habrá tantos valores como
# estos como tantos documentos existan que tengan una representación vectorial con igual dirección y sentido

if not np.array_equal(np.argmax(similarities, axis=1), idxs):
    # Si el software llega a este punto, significa que existe al menos un documento
    # cuya representación vectorial está alineada con la representación del elemento
    # con el que se está comparando pero que no es el documento de referencia
    # Sólo para no complicar el software, considerando que la probabilidad de que esto
    # suceda es muy pequeña, la forma de resolver esta contingencia es pedir una
    # nueva ejecución
    print("Por favor, vuelva a ejecutar la celda")
    raise ValueError

In [25]:
from colorama import Fore, Back, Style

max_similarities_ids = np.argsort(similarities, axis=1)[:, -(N_TOP_SCORE+1):-1]
doc_similarities = similarities[np.arange(
    N_DOCS_TO_COMPARE)[:, None], max_similarities_ids]

original_targets = newsgroups_train.target[idxs]
max_similarity_targets = newsgroups_train.target[max_similarities_ids]

target_names = np.array(newsgroups_train.target_names)

for i in range(N_DOCS_TO_COMPARE):
    print(Back.MAGENTA + 'Top-{0} de similaridad de documento id {1}'.format(
        N_TOP_SCORE, idxs[i]) + Back.RESET)

    columns = zip(range(N_TOP_SCORE), doc_similarities[i], np.repeat(
        target_names[original_targets[i]], N_TOP_SCORE),
        target_names[max_similarity_targets[i]])

    print(tabulate(columns, headers=['#', 'Similarity', 'Reference Target', 'Max similarity Target']))

    print()


Top-5 de similaridad de documento id 8687
  #    Similarity  Reference Target    Max similarity Target
---  ------------  ------------------  -----------------------
  0     0.0908759  rec.motorcycles     alt.atheism
  1     0.0946011  rec.motorcycles     sci.med
  2     0.0959232  rec.motorcycles     sci.space
  3     0.114874   rec.motorcycles     rec.autos
  4     0.135036   rec.motorcycles     rec.autos

Top-5 de similaridad de documento id 8010
  #    Similarity  Reference Target    Max similarity Target
---  ------------  ------------------  -----------------------
  0      0.229822  comp.windows.x      comp.graphics
  1      0.274401  comp.windows.x      comp.windows.x
  2      0.28986   comp.windows.x      comp.windows.x
  3      0.297142  comp.windows.x      comp.windows.x
  4      0.363557  comp.windows.x      comp.windows.x

Top-5 de similaridad de documento id 9375
  #    Similarity  Reference Target    Max similarity Target
---  ------------  ------------------  ----------

*Conclusión 1*: Como se puede ver en la mayoria de los casos las clases relacionadas a los documentos con mayor similaridad coinciden, son parecidos o están dentro de la misma temática que aquellas de las referencias. Interesante también es notar que para los primeros casos en los que la similaridad es menor a 0.1, se ven diferencias sustanciales entre las clases de los documentos (por ejemplo: rec.motorcycles vs alt.atheism).

A continuación, se analizará en detalle el último caso (#4) para ver si existe similaridad entre los documentos.

In [26]:
SELECTED_DOCUMENT = 4
TOP_CASE = 4

print('Reference document:', idxs[SELECTED_DOCUMENT])
print('-------------------')
print(newsgroups_train.data[idxs[SELECTED_DOCUMENT]])
print('------------------')
print('Compared document:', max_similarities_ids[SELECTED_DOCUMENT, TOP_CASE])
print('------------------')
print(newsgroups_train.data[max_similarities_ids[SELECTED_DOCUMENT, TOP_CASE]])

Reference document: 7511
-------------------
I'm using an OAK based VGA card on my computer (640x480x256).  I've
downloaded the driver from ftp.cica.indiana.edu and I've had good luck
with it.  However, does anyone know if a faster driver is available for
this card?

Thanks,

Mike
------------------
Compared document: 137
------------------
Hi, I'm looking for the 3-D studio driver for the
	Oak card with 1 M of RAM.
	This would be GREATLY (and I mean that) appreciated


*Conclusión 2*: Como puede verse, si bien estos documentos no versan sobre especificamente el mismo tema, hablan sobre cuestiones muy relacionadas, específicamente de temas de hardware y software gráfico de computadoras.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

from tqdm import tqdm

configs = [{'vectorizer': TfidfVectorizer(lowercase=False), 'model': MultinomialNB()},
           {'vectorizer': TfidfVectorizer(), 'model': MultinomialNB(alpha=0.5)},
           {'vectorizer': CountVectorizer(), 'model': ComplementNB()},
           {'vectorizer': TfidfVectorizer(), 'model': ComplementNB()},
           {'vectorizer': TfidfVectorizer(), 'model': ComplementNB(norm=True)}]

configsScores = []

In [28]:
y_train = newsgroups_train.target

for i in tqdm(range(len(configs))):
    config = configs[i]

    vectorizer = config['vectorizer']

    X_train = vectorizer.fit_transform(newsgroups_train.data)

    model = config['model']
    model.fit(X_train, y_train)

    X_test = vectorizer.transform(newsgroups_test.data)
    y_test = newsgroups_test.target

    y_pred = model.predict(X_test)
    score = f1_score(y_test, y_pred, average='macro')

    configsScores.append([config['vectorizer'], config['model'], score]) 

print(tabulate(configsScores, headers=['Vectorizador', 'Modelo', 'F1 Score']))

100%|██████████| 5/5 [00:19<00:00,  3.98s/it]

Vectorizador                      Modelo                      F1 Score
--------------------------------  ------------------------  ----------
TfidfVectorizer(lowercase=False)  MultinomialNB()             0.563325
TfidfVectorizer()                 MultinomialNB(alpha=0.5)    0.615342
CountVectorizer()                 ComplementNB()              0.637427
TfidfVectorizer()                 ComplementNB()              0.692953
TfidfVectorizer()                 ComplementNB(norm=True)     0.689778


*Conclusión*: Se probaron diferentes vectorizadores, modelos y parámetros para los mismos en ciclos de entrenamiento. Se observa que como es de esperarse, considerar las mayúsuclas y minúsculas hace que el modelo performe peor. Entre otras cosas, se ve que la performance del modelo ComplementNB no mejora con la normalización de los pesos. Por último, se nota que el vectorizador CountVectorizer ofrece una peor performance que el TfidfVectorizer para el mismo tipo de modelo en un escenario de entrenamiento como el desarrollado.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.


In [29]:
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [30]:
def getTermFromId(id):
    key_list = list(tfidfvect.vocabulary_.keys())
    val_list = list(tfidfvect.vocabulary_.values())
    
    return key_list[val_list.index(id)]

In [31]:
N_TERMS_TO_COMPARE = 5
N_TOP_SCORE = 5

XT_train = np.transpose(X_train)

# Para este punto se utilizan IDs elegidos desde el vocabulario para evitar
# trabajar con términos que no sean fáciles de relacionar. Los términos elegidos
# son: 'surfaces', 'day', 'separate', 'early', 'prepared'
idxs = [86591, 31990, 81658, 35974, 72972]

similarities = cosine_similarity(XT_train[idxs], XT_train)

In [32]:
from colorama import Fore, Back, Style

max_similarities_ids = np.argsort(similarities, axis=1)[:, -(N_TOP_SCORE+1):]

for i in range(N_TERMS_TO_COMPARE):
    print('Top-' + str(N_TOP_SCORE) + ' de similaridad del término '
          + Back.MAGENTA + getTermFromId(idxs[i]) + Back.RESET)

    simil_ids = max_similarities_ids[i]

    # Aquí se utiliza otra estrategia para eliminar el id del término que se 
    # está tomando de referencia de los ids ordenados por similaridad, ya que
    # en el caso de los términos parece ser que es más probable que haya valores
    # de similaridad teóricamente iguales a 1 
    if idxs[i] in simil_ids:
        np.delete(simil_ids, np.where(simil_ids == idxs[i]))
    else:
        simil_ids = simil_ids[1:]

    similarityTerms = [getTermFromId(similId) for similId in simil_ids]
    similarities[i, simil_ids]

    doc_similarities = similarities[np.arange(
        N_DOCS_TO_COMPARE)[:, None], max_similarities_ids]

    print(tabulate(zip(range(N_DOCS_TO_COMPARE), similarities[i, simil_ids], similarityTerms),
                   headers=['#', 'Similarity', 'Related term']))

    print()

Top-5 de similaridad del término surfaces
  #    Similarity  Related term
---  ------------  --------------
  0      0.325171  planetary
  1      0.410352  schroeder
  2      0.410476  marching
  3      0.485746  stereos
  4      0.526583  deimos

Top-5 de similaridad del término day
  #    Similarity  Related term
---  ------------  --------------
  0      0.192257  henneman
  1      0.195412  the
  2      0.197666  pinchrunner
  3      0.216522  specificed
  4      0.216522  scotyy

Top-5 de similaridad del término separate
  #    Similarity  Related term
---  ------------  --------------
  0      0.262995  desktops
  1      0.291505  reflected
  2      0.298079  deceiving
  3      0.335609  unjustified
  4      0.344676  polocies

Top-5 de similaridad del término early
  #    Similarity  Related term
---  ------------  --------------
  0      0.145333  reprinting
  1      0.145612  phases
  2      0.162035  consulted
  3      0.207611  derrick
  4      0.207611  standout

Top-5 de s

*Conclusión*: Del análisis realizado sobre los datos anteriores, no se encuentran grandes similaridades semánticas entre los términos analizados, aún en los casos en los que este indicador toma valores mayores a 0.5. Si bien se podría esperar, en términos teóricos, que surjan sinónimos de los términos elegidos, este no ha sido el caso para el set de términos seleccionados. 